In [1]:
!cat deployment.yaml

apiVersion: apps/v1beta2
kind: Deployment
metadata:
  name: tomcat-deployment
spec:
  selector:
    matchLabels:
      app: tomcat
  replicas: 1
  template:
    metadata:
      labels:
        app: tomcat
    spec:
      containers:
      - name: tomcat
        image: tomcat:9.0
        ports:
        - containerPort: 8080

#### we can see, only 1 replica is mentioned. Hence we have one container in the pod.

In [2]:
!kubectl get pods

NAME                                 READY     STATUS    RESTARTS   AGE
hastlecast-74f7dc47d8-5gxfs          1/1       Running   0          3m
tomcat-deployment-56ff5c79c5-ljvdb   1/1       Running   0          36m


In [3]:
!kubectl scale --replicas=4 deployment/tomcat-deployment

deployment.extensions/tomcat-deployment scaled


In [4]:
!kubectl get pods

NAME                                 READY     STATUS    RESTARTS   AGE
hastlecast-74f7dc47d8-5gxfs          1/1       Running   0          3m
tomcat-deployment-56ff5c79c5-d8xhs   1/1       Running   0          11s
tomcat-deployment-56ff5c79c5-ksxjr   1/1       Running   0          11s
tomcat-deployment-56ff5c79c5-ljvdb   1/1       Running   0          37m
tomcat-deployment-56ff5c79c5-lsk9h   1/1       Running   0          11s


In [10]:
!kubectl describe pod tomcat-deployment

Name:           tomcat-deployment-56ff5c79c5-d8xhs
Namespace:      default
Node:           minikube/10.0.2.15
Start Time:     Tue, 11 Sep 2018 14:56:59 +0200
Labels:         app=tomcat
                pod-template-hash=1299173571
Annotations:    <none>
Status:         Running
IP:             172.17.0.7
Controlled By:  ReplicaSet/tomcat-deployment-56ff5c79c5
Containers:
  tomcat:
    Container ID:   docker://be3400257ddefcadc2ca1e790770c94cbf07bd2b99c42e149b2ca8b464c7d9a4
    Image:          tomcat:9.0
    Image ID:       docker-pullable://tomcat@sha256:c9475761c26b74f7d92864f32433eba21f3718a3be9be5df3e24e2d564740707
    Port:           8080/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Tue, 11 Sep 2018 14:57:01 +0200
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-qh4bq (ro)
Conditions:
  Type           Status
  Initialized    True 
  Ready       

#### Change access method from NodePort to LoadBalancer

In [9]:
!kubectl expose deployment tomcat-deployment --type=LoadBalancer \
--port=8080 --target-port=8080 --name tomcat-load-balancer

service/tomcat-load-balancer exposed


In [12]:
!kubectl describe pod tomcat-deployment

Name:           tomcat-deployment-56ff5c79c5-d8xhs
Namespace:      default
Node:           minikube/10.0.2.15
Start Time:     Tue, 11 Sep 2018 14:56:59 +0200
Labels:         app=tomcat
                pod-template-hash=1299173571
Annotations:    <none>
Status:         Running
IP:             172.17.0.7
Controlled By:  ReplicaSet/tomcat-deployment-56ff5c79c5
Containers:
  tomcat:
    Container ID:   docker://be3400257ddefcadc2ca1e790770c94cbf07bd2b99c42e149b2ca8b464c7d9a4
    Image:          tomcat:9.0
    Image ID:       docker-pullable://tomcat@sha256:c9475761c26b74f7d92864f32433eba21f3718a3be9be5df3e24e2d564740707
    Port:           8080/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Tue, 11 Sep 2018 14:57:01 +0200
    Ready:          True
    Restart Count:  0
    Environment:    <none>
    Mounts:
      /var/run/secrets/kubernetes.io/serviceaccount from default-token-qh4bq (ro)
Conditions:
  Type           Status
  Initialized    True 
  Ready       

In [21]:
!kubectl describe services tomcat-load-balancer

Name:                     tomcat-load-balancer
Namespace:                default
Labels:                   app=tomcat
Annotations:              <none>
Selector:                 app=tomcat
Type:                     LoadBalancer
IP:                       10.104.114.116
Port:                     <unset>  8080/TCP
TargetPort:               8080/TCP
NodePort:                 <unset>  30810/TCP
Endpoints:                172.17.0.4:8080,172.17.0.6:8080,172.17.0.7:8080 + 1 more...
Session Affinity:         None
External Traffic Policy:  Cluster
Events:                   <none>


<font color=red>We can  see load balancer has a public ip assigned</font>

In [23]:
!kubectl get deployments

NAME                DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
hastlecast          1         1         1            1           15m
tomcat-deployment   4         4         4            4           49m


In [26]:
!kubectl rollout status deployment tomcat-deployment

deployment "tomcat-deployment" successfully rolled out


In [27]:
!kubectl set image deployment/tomcat-deployment tomcat=tomcat:9.0.1

deployment.extensions/tomcat-deployment image updated


In [30]:
!kubectl rollout history deployment/tomcat-deployment

deployments "tomcat-deployment"
REVISION  CHANGE-CAUSE
1         <none>
2         <none>



In [31]:
!kubectl rollout history deployment/tomcat-deployment --revision=2

deployments "tomcat-deployment" with revision #2
Pod Template:
  Labels:	app=tomcat
	pod-template-hash=138885474
  Containers:
   tomcat:
    Image:	tomcat:9.0.1
    Port:	8080/TCP
    Host Port:	0/TCP
    Environment:	<none>
    Mounts:	<none>
  Volumes:	<none>

